In [55]:
# 도구 임포트
import torch
import torch.nn as nn

In [56]:
# 임의의 텐서 만들기 (1*1*28*28)

inputs = torch.Tensor(1,1,28,28)
print('inputs 텐서의 크기: {}'.format(inputs.shape))

inputs 텐서의 크기: torch.Size([1, 1, 28, 28])


In [57]:
# 첫번째 합성곱층 선언
conv1 = nn.Conv2d(in_channels = 1,      # 입력받을 채널
                  out_channels = 32,    # 내보낼 채널
                  kernel_size = 3,      # 커널 사이즈(3,3)
                  padding=1)            # 패딩사이즈(zero padding)

print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [58]:
# 두번째 합성곱층 구현
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [59]:
# 풀링 선언
pool = nn.MaxPool2d(2)      # 인자를 하나만 넣으면 kernel, stride로 모두 지정된다.
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [60]:
# 구현한 것들을 연결하여 모델 만들기

# 첫번째 합성곱을을 통과한 inputs의 변화 보기
out = conv1(inputs)
print(out.shape)
# inputs 텐서의 크기: torch.Size([1, 1, 28, 28])
# >> torch.Size([1, 32, 28, 28])
# 32채널 : conv1의 아웃채널을 32로 지정했기때문
# 28,28 : padding 을 1로 설정하고 kernel size가 3*3이기 때문에 크기가 보존 됨

torch.Size([1, 32, 28, 28])


In [61]:
# maxpool을 통과한 값도 확인
# MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [62]:
# conv2 를 통과한 사이즈도 확인
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [63]:
# 다시 pooling 통과한 크기 확인
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [64]:
# 결과로 나온 feature map을 1차원으로 펴주자
# n번째 차원에 접근하게 해주는 .size(n)를 사용하자

# out의 각 차원이 몇인지 확인
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [65]:
# 이 값을 갖고 .view()를 사용해 텐서를 펼쳐보자  

# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [66]:
# 펼친 feature map을 fully connected layer에 통과시키자

fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


---
### 위 방법들을 이용해 CNN으로 MNIST 분류하기

In [67]:
# 필요한 도구 임포트
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [68]:
# 사용할 디바이스 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# gpu 사용 가능일 때에도 cuda.manual_seed_all 지정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [69]:
# 학습에 사용할 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [70]:
# DataLoader에 사용하기 위한 Dataset을 정의
mnist_train = dsets.MNIST(root = 'MNIST_data/', # 다운받을 경로
                          train = True, # True를 지정하면 train_data를 다운로드
                          transform = transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [71]:
# DataLoader사용하여 batch_size를 지정
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [72]:
# 클래스로 모델을 설계

class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()

        # 첫번째 층 || input_image(?, 28, 28, 1) > Conv1 > (?, 28, 28, 32) > pool > (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 두번째 층 || (?, 14, 14, 32) > conv2 > (?, 14, 14, 64) > pool > (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # Fully conected layer || 7*7*64 inputs > 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층을 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [73]:
# CNN 모델 정의
model = CNN().to(device)

In [74]:
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 sofrmax가 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [75]:
# 총 배치수 확인
total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

# 총 배치의 수(600) * 배치 사이즈(100) = 전체 훈련 데이터(60,000)

총 배치의 수: 600


In [76]:
# training_epochs 만큼 훈련 진행

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:    # 미니 배치 단위로 가져오기. x:데이터, y:레이블
        # 이미지는 이미 (28*28)이므로 reshape 필요 없음
        # 라벨은 one-hot encoding 되지 않았음
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225616753
[Epoch:    2] cost = 0.0629506782
[Epoch:    3] cost = 0.0463600233
[Epoch:    4] cost = 0.0375382006
[Epoch:    5] cost = 0.031426914
[Epoch:    6] cost = 0.0261538494
[Epoch:    7] cost = 0.0220672395
[Epoch:    8] cost = 0.0183010641
[Epoch:    9] cost = 0.0164200179
[Epoch:   10] cost = 0.0135397073
[Epoch:   11] cost = 0.0104772886
[Epoch:   12] cost = 0.0100616114
[Epoch:   13] cost = 0.00816558022
[Epoch:   14] cost = 0.00662278617
[Epoch:   15] cost = 0.00705707865


In [78]:
# 테스트를 진행

# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_predict = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_predict.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\lemon\anaconda3\envs\chch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\lemon\anaconda3\envs\chch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9860999584197998
